Import packages and processed run metrics

In [96]:
# packages
import re
import numpy as np
import pandas as pd

# control
project_names = [
    'decay--sst2',
    'decay--pubmed',
    'decay--cifar--alpha5.0',
    'decay--cifar--alpha0.5',
    'decay--cifar--alpha0.1'
]
validation_metric = 'Results/val_acc'

# read each data set
df = []
for project_name in project_names:
    temp = pd.read_csv(f'{project_name}.csv')
    print(temp.shape)
    df.append(temp)

df = pd.concat(df, axis=0, ignore_index=True)
print()
print('all runs:', df.shape)

(1201, 36)
(780, 28)
(560, 27)
(520, 27)
(520, 27)

all runs: (3581, 37)


Write function to return summaries of run's metric performance

In [97]:
def process_run_metrics(filtered_df, method='exact'):
    pass


    ## Process metrics and get top runs for each
    # are large or small metric values are desirable?
    descending_metrics = [name for name in filtered_df.columns if re.search('Results.*test', name)]
    ascending_metrics = [
        descending_metrics.pop(descending_metrics.index(name))
        for name in descending_metrics
        if re.search('std', name)
    ]
    print('metrics:')
    print('\t', descending_metrics)
    print('\t', ascending_metrics)

    # convert metrics to numeric
    #filtered_df[descending_metrics + ascending_metrics] = filtered_df[descending_metrics + ascending_metrics].apply(pd.to_numeric)

    # all non-metrics columns are used to identify the experimental run
    filtered_runs = filtered_df[[
        name for name in filtered_df.columns
        if name not in descending_metrics + ascending_metrics
    ]]

    # rank the metrics
    ranked_descending = filtered_df.groupby(['dataset'])[descending_metrics].rank(
        method='min',
        ascending=False
    )
    ranked_ascending = filtered_df.groupby(['dataset'])[ascending_metrics].rank(
        method='min',
        ascending=True
    )

    # combine and sort the ranked_metrics
    ranked_metrics = pd.concat([ranked_descending, ranked_ascending], axis=1)
    ranked_metrics = ranked_metrics[sorted(ranked_metrics.columns)]
    filtered_ranks = filtered_runs.join(ranked_metrics)


    ## Manipulate rank data to be summarized by runs and metrics
    # convert to long format
    # filter to top ranks
    long_filtered_ranks = pd.melt(filtered_ranks, id_vars=filtered_runs.columns, var_name='metric')
    top_filtered_metrics = long_filtered_ranks.loc[long_filtered_ranks.value <= 3].copy()  # top 3 runs

    # compute rank summaries to understand what runs are top overall
    top_filtered_metrics['rank_one_ind'] = (top_filtered_metrics.value == 1)
    top_filtered_metrics['rank_two_ind'] = (top_filtered_metrics.value == 2)
    top_filtered_metrics['rank_three_ind'] = (top_filtered_metrics.value == 3)
    top_filtered_metrics.replace(False, pd.NA, inplace=True)

    # summarized metric ranks for run type
    rank_summary_columns = ['rank_one_ind', 'rank_two_ind', 'rank_three_ind', 'value']
    #id_columns = ['dataset', 'method', 'finetune']
    id_columns = ['method', 'finetune']
    run_summary = top_filtered_metrics.groupby(id_columns)[rank_summary_columns].count()

    # summarize metric ranks for metric choice
    metric_summary = top_filtered_metrics.loc[top_filtered_metrics.method == method]
    metric_summary = metric_summary.sort_values(by='metric').groupby('metric')[rank_summary_columns].count()

    return (
        run_summary,
        metric_summary,
        filtered_runs
    )


Filter data to runs of interest
Apply summary function to filtered dataset

In [0]:
subset_df = df
print('input dataset:', df.shape)

## row (run) filtering
# remove extra hyper-parameter searches
subset_df = subset_df.loc[(df.n_epochs < 6)]
print('\t reduce to 3 or fewer local update steps:', df.shape)
subset_df = subset_df.loc[(df.K < 6) | (df.method != 'pfedme')]
print('\t reduce to 3 or fewer local meta-learning steps for pfedme:', df.shape)
subset_df = subset_df.loc[(int(10 * df.beta) % 2 == 0) | (df.method != 'exact')]
print('\t reduce beta grid for decay:', df.shape)

"""
subset_df = subset_df.loc[[
    True if beta in betas else False
    for beta in subset_df.beta
    ] | (subset_df.method != 'exact')
]

# column (metric) filtering
subset_df = subset_df[[
    name for name in subset_df.columns
    if not re.search('f1', name)
       and not re.search('loss', name)
]]
"""

In [0]:
## get best runs for each group
# regardless of finetuning
best_runs = df.loc[
    df.groupby(['dataset', 'method', 'finetune']) \
    [validation_metric].idxmax()  # return index of max validation metric
]

# treat finetuning groups as seperate
ft_yes = best_runs.loc[best_runs.finetune == 1]
ft_no = best_runs.loc[best_runs.finetune == 0]

For all best runs, regardless of finetuning, produce summaries

In [98]:
(all_run_summary, all_metric_summary, _) = process_run_metrics(best_runs, method='exact')

# run summary
print(all_run_summary.shape)
all_run_summary.sort_values(by='value', ascending=False)

input runs: (2610, 21)
top validation runs: (44, 21)
metrics:
	 ['Results/test_acc', 'Results/test_acc_bottom_decile', 'Results_unseen/test_acc', 'Results_unseen/test_acc_bottom_decile', 'Results_weighted/test_acc', 'Results_weighted_unseen/test_acc']
	 ['Results/test_acc_std', 'Results_unseen/test_acc_std']


,Unnamed: 0,method,finetune,dataset,n_epochs,batch_size,lr,beta,regular_weight,K,finetune_lr,Results/val_acc,alpha
3399,338,ditto,0,cifar--alpha0.1,3,NaN,0.500,NaN,0.50,NaN,NaN,0.487736,0.1
3548,487,exact,0,cifar--alpha0.1,3,NaN,0.100,0.2,NaN,NaN,NaN,0.566214,0.1
3373,312,fedavg,0,cifar--alpha0.1,3,NaN,0.050,NaN,NaN,NaN,NaN,0.553132,0.1
3351,290,fedbn,0,cifar--alpha0.1,3,NaN,0.100,NaN,NaN,NaN,NaN,0.407272,0.1
3353,292,fedbn,1,cifar--alpha0.1,3,NaN,0.010,NaN,NaN,NaN,NaN,0.349454,0.1
3332,271,fedem,0,cifar--alpha0.1,3,NaN,0.050,NaN,NaN,NaN,NaN,0.563560,0.1
3198,137,pfedme,0,cifar--alpha0.1,3,NaN,0.500,NaN,0.05,NaN,NaN,0.581741,0.1
2873,332,ditto,0,cifar--alpha0.5,3,NaN,0.100,NaN,0.10,NaN,NaN,0.688934,0.5
3059,328,exact,0,cifar--alpha0.5,1,NaN,0.050,0.0,NaN,NaN,NaN,0.712604,0.5
2869,328,fedavg,0,cifar--alpha0.5,1,NaN,0.050,NaN,NaN,NaN,NaN,0.712604,0.5


View processed summaries

In [100]:
print(all_metric_summary.shape)

# metrics summary
sort_by = ['value', 'rank_one_ind', 'rank_two_ind', 'rank_three_ind']
all_metric_summary.sort_values(by=sort_by, ascending=False)

(8, 4)


,rank_one_ind,rank_two_ind,rank_three_ind,value
metric,,,,
Results_unseen/test_acc,3,1,1,5
Results_weighted_unseen/test_acc,3,1,1,5
Results/test_acc_bottom_decile,2,3,0,5
Results/test_acc_std,4,0,0,4
Results/test_acc,2,2,0,4
Results_unseen/test_acc_std,1,2,1,4
Results_unseen/test_acc_bottom_decile,2,0,1,3
Results_weighted/test_acc,1,2,0,3
